In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys

sys.path.append('./helpers_models/')
sys.path.append('./data_visualization_and_augmentations/')
sys.path.append('../torch_videovision/')
sys.path.append('./important_csvs/')

from helpers_resnet import *

resnet = torchvision.models.resnet50(pretrained=True)
adaptive_pooling = AdaptiveConcatPool2d()
head = Head()
resnet.avgpool = adaptive_pooling
resnet.fc = head

os.environ['CUDA_VISIBLE_DEVICES']='0,1,2'

resnet = resnet.cuda()

for param in resnet.parameters():
    param.requires_grad = False
    
for param in resnet.avgpool.parameters():
    param.requires_grad = True
    
for param in resnet.fc.parameters():
    param.requires_grad = True

resnet = nn.DataParallel(resnet)
check_freeze(resnet.module)

In [ ]:
from helpers_thresholds import *

In [ ]:
plt.rcParams['figure.figsize'] = (12,6)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 40}
plt.rc('font', **font)

In [ ]:
tensor_transform = get_tensor_transform('ImageNet', False)
train_spat_transform = get_spatial_transform(2)
train_temp_transform = get_temporal_transform()
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

root_dir = '/media/scratch/astamoulakatos/nsea_video_jpegs/'
df = pd.read_csv('./small_dataset_csvs/events_with_number_of_frames_stratified.csv')
# df_train = get_df(df, 20, True, False, False)
# class_image_paths, end_idx, idx_label= get_indices(df_train, root_dir)
# train_loader = get_loader(1, 128, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, False, True, True, 1)
df_valid = get_df(df, 20, False, True, False)
class_image_paths, end_idx, idx_label = get_indices(df_valid, root_dir)
valid_loader = get_loader(1, 50, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, True, True, 1)
df_test = get_df(df, 20, False, False, True)
class_image_paths, end_idx, idx_label = get_indices(df_test, root_dir)
test_loader = get_loader(1, 50, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, True, True, 1)

torch.cuda.empty_cache()

In [ ]:
load = True
if load:
    checkpoint = torch.load('/media/scratch/astamoulakatos/saved-resnet-models/fifth-small-aug-final/best-checkpoint-010epoch.pth')
    resnet.load_state_dict(checkpoint['model_state_dict'])
    print('loading pretrained freezed model!')

In [ ]:
device = torch.device('cuda')

In [ ]:
from predictions import *

In [ ]:
show_batch(valid_loader, 4)

In [ ]:
plot_predictions_actuals(valid_loader, 10, resnet, device)

In [ ]:
resnet.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for X, y in valid_loader:
        X = X.to(device)
        y = Variable(y.float()).to(device) 
        #X = X.permute(0,2,1,3,4)
        y = y.squeeze(dim=1)
        y = y.float()
        output, _ = resnet(X)
        y = y.detach().cpu()
        #loss = criterion(output, y)
        preds = torch.sigmoid(output)
        preds = preds.to(torch.float32)
        preds = preds.detach().cpu()
        y_pred.append(preds)
        y_true.append(y)

In [ ]:
from helpers_thresholds import *

In [ ]:
y_pr = torch.cat(y_pred)
y_tr = torch.cat(y_true)

In [ ]:
y_pr = y_pr.numpy()
y_tr = y_tr.numpy()

In [ ]:
classes = ['Exposure', 'Burial', 'Field Joint', 'Anode', 'Free Span']

In [ ]:
plot_pr_curves(0.0, y_tr, y_pr, classes, '_2d_full_size')

In [ ]:
thresholds = nsea_compute_thresholds(y_tr, y_pr, classes)

In [ ]:
thresholds

In [ ]:
df_thresholds = pd.DataFrame({'Threshold': thresholds}).T
df_thresholds

# Table 1

In [ ]:
print(df_thresholds.to_latex(float_format=lambda x: '%.3f' % truncate_decimals(x,3)))

## validation set

In [ ]:
result_valid = new_compute_metrics(y_tr, y_pr, thresholds, classes)

In [ ]:
result_valid

In [ ]:
print(result_valid.to_latex(float_format=lambda x: '%.3f' % truncate_decimals(x,3)))

In [ ]:
acc, f1 = compute_label_metrics(y_tr, y_pr, 0.5, classes)

In [ ]:
acc

In [ ]:
f1

## test set

In [ ]:
resnet.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(device)
        y = Variable(y.float()).to(device) 
        #X = X.permute(0,2,1,3,4)
        y = y.squeeze(dim=1)
        y = y.float()
        output = resnet(X)
        y = y.detach().cpu()
        #loss = criterion(output, y)
        preds = torch.sigmoid(output)
        preds = preds.to(torch.float32) 
        preds = preds.detach().cpu()
        y_pred.append(preds)
        y_true.append(y)

In [ ]:
y_tr = np.vstack([t.__array__() for tensor in y_true for t in tensor])

In [ ]:
y_pr = np.vstack([t.__array__() for tensor in y_pred for t in tensor])

In [ ]:
result_test = new_compute_metrics(y_tr, y_pr, thresholds, classes)

In [ ]:
result_test

In [ ]:
print(result_test.to_latex(float_format=lambda x: '%.3f' % truncate_decimals(x,3)))

In [ ]:
precision, recall, f1, _ = precision_recall_fscore_support(y_tr, y_pr)